In [24]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
import random
from collections import Counter

from generator import generateCliffordCircuit
# from execution.transpiler import transpileListToQiskitCircuit

In [25]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [26]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['X', 'I', 'S', 'X', 'I'], ['I', 'CNOT_T', 'I', 'I', 'CNOT_C'], ['S', 'X', 'I', 'S', 'Z'], ['I', 'I', 'CNOT_T', 'CNOT_C', 'I'], ['S', 'S', 'I', 'S', 'Z']]
     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ X ├─░───────░─┤ S ├─░───────░─┤ S ├
     ├───┤ ░ ┌───┐ ░ ├───┤ ░       ░ ├───┤
q_1: ┤ I ├─░─┤ X ├─░─┤ X ├─░───────░─┤ S ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░ ┌───┐ ░ ├───┤
q_2: ┤ S ├─░───┼───░─┤ I ├─░─┤ X ├─░─┤ I ├
     ├───┤ ░   │   ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_3: ┤ X ├─░───┼───░─┤ S ├─░───■───░─┤ S ├
     ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤
q_4: ┤ I ├─░───■───░─┤ Z ├─░───────░─┤ Z ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [27]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    if type(pauli) == qk.quantum_info.Pauli:
        p = pauli
    elif type(pauli) == str:
        p = qk.quantum_info.Pauli(pauli)

    return p.evolve(cliff,frame="s")


In [ ]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^dag, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    if type(pauli) == qk.quantum_info.Pauli:
        p = pauli
    elif type(pauli) == str:
        p = qk.quantum_info.Pauli(pauli)

    for 

    return p.evolve(cliff,frame="s")

In [28]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('XYZIX')

In [29]:
def split_circuit_by_barrier(circuit):
    qasm = circuit.qasm()
    prelude = []
    circuits = [[]]
    for line in qasm.splitlines():
        if any([line.startswith(t) for t in ['OPENQASM', 'include', 'qreg', 'creg']]):
            prelude.append(line)
        elif line.startswith('barrier'):
            circuits.append([])
        else:
            circuits[-1].append(line)
    circuits_with_prelude = [prelude+circuit for circuit in circuits]
    for circuit_with_prelude in circuits_with_prelude:
        yield qk.QuantumCircuit.from_qasm_str('\n'.join(circuit_with_prelude))

# for subcircuit in (split_circuit_by_barrier(qiskitCircuit)):
#     print(subcircuit)

In [30]:
def G_twirling(circuit):
    '''This function performs G-twirling on a circuit. It randomly chooses a Pauli from {X,Y,Z,I} and prepends it before a layer, then appends P'=CPC^T after the layer.}
    Inputs: circuit - A Qiskit Circuit object.
    Outputs: new_circuit - A Qiskit Circuit object.'''
    num_qubits = circuit.num_qubits
    new_circuit = qk.QuantumCircuit(num_qubits)
    for index, subcircuit in enumerate(split_circuit_by_barrier(circuit)):
        # if len(subcircuit.data) == 1:
        #     operated_qubits = [subcircuit.find_bit(q).index for q in subcircuit.data[0].qubits]
        # else:
        #     operated_qubits = list(range(num_qubits))
        pauli_str = ''.join(random.choice(["X","Y","Z","I"]) for _ in range(num_qubits))
        pauli = qk.quantum_info.Pauli(pauli_str)
        new_circuit = new_circuit.compose(pauli,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(subcircuit,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(Clifford_Permute(subcircuit,pauli).to_instruction(),qubits = range(num_qubits))
        if index != len(list(split_circuit_by_barrier(circuit)))-1:
            new_circuit.barrier()
    return new_circuit


In [31]:
G_twirling(qiskitCircuit).draw()

┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐     »
q_0: ┤0              ├┤ X ├┤0        ├─░─┤0              ├─────»
     │               │├───┤│         │ ░ │               │┌───┐»
q_1: ┤1              ├┤ I ├┤1        ├─░─┤1              ├┤ X ├»
     │               │├───┤│         │ ░ │               │└─┬─┘»
q_2: ┤2 Pauli(IXZYZ) ├┤ S ├┤2 -IXZYZ ├─░─┤2 Pauli(ZZZIY) ├──┼──»
     │               │├───┤│         │ ░ │               │  │  »
q_3: ┤3              ├┤ X ├┤3        ├─░─┤3              ├──┼──»
     │               │├───┤│         │ ░ │               │  │  »
q_4: ┤4              ├┤ I ├┤4        ├─░─┤4              ├──■──»
     └───────────────┘└───┘└─────────┘ ░ └───────────────┘     »
«     ┌───────────────┐ ░ ┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐»
«q_0: ┤0              ├─░─┤0              ├┤ S ├┤0        ├─░─┤0              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_1: ┤1              ├─░─┤1              ├┤ X ├┤1        ├─░─┤1              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_2: ┤2 Pauli(ZZZIY) ├─░─┤2 Pauli(ZYIII) ├┤ I ├┤2 -ZXIII ├─░─┤2 Pauli(ZZXZY) ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_3: ┤3              ├─░─┤3              ├┤ S ├┤3        ├─░─┤3              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_4: ┤4              ├─░─┤4              ├┤ Z ├┤4        ├─░─┤4              ├»
«     └───────────────┘ ░ └───────────────┘└───┘└─────────┘ ░ └───────────────┘»
«          ┌───────────────┐ ░ ┌───────────────┐┌───┐┌─────────┐
«q_0: ─────┤0              ├─░─┤0              ├┤ S ├┤0        ├
«          │               │ ░ │               │├───┤│         │
«q_1: ─────┤1              ├─░─┤1              ├┤ S ├┤1        ├
«     ┌───┐│               │ ░ │               │├───┤│         │
«q_2: ┤ X ├┤2 Pauli(ZZXZY) ├─░─┤2 Pauli(ZYIXI) ├┤ I ├┤2 -ZXIYI ├
«     └─┬─┘│               │ ░ │               │├───┤│         │
«q_3: ──■──┤3              ├─░─┤3              ├┤ S ├┤3        ├
«          │               │ ░ │               │├───┤│         │
«q_4: ─────┤4              ├─░─┤4              ├┤ Z ├┤4        ├
«          └───────────────┘ ░ └───────────────┘└───┘└─────────┘

In [32]:
circuit_ensemble = [G_twirling(qiskitCircuit) for _ in range(10)]
# for circ in circuit_ensemble:
#     print(circ)

In [33]:
def random_choices_with_counts(items, n):
    """
    Randomly choose with replacement n items from a list and return a dictionary
    with the counts of each chosen item.
    
    Parameters:
        items (list): List of items to choose from.
        n (int): Number of items to choose.
    
    Returns:
        dict: Dictionary with the counts of each chosen item.
    """
    chosen_items = random.choices(items, k=n)
    counts = Counter(chosen_items)
    return dict(counts)

In [34]:
def find_circuit_from_name(circuit_list,target_name):
    # Find the circuit with the given name
    found_circuit = None
    for circuit in circuit_list:
        if circuit.name == target_name:
            found_circuit = circuit
            break

    # Check if the circuit was found
    if found_circuit is not None:
        # print(f"Circuit with name '{target_name}' found:")
        # print(found_circuit)
        return found_circuit
    else:
        print(f"No circuit with name '{target_name}' found.")
        return found_circuit

In [35]:
me = qk.QuantumCircuit(5)
me.pauli(pauli_string = "XX",qubits = [1,3])
print(me)

                   
q_0: ──────────────
     ┌────────────┐
q_1: ┤0           ├
     │            │
q_2: ┤  Pauli(XX) ├
     │            │
q_3: ┤1           ├
     └────────────┘
q_4: ──────────────
                   


In [36]:
num_experiments = 1000
freq_dict = random_choices_with_counts([circ.name for circ in circuit_ensemble],num_experiments)
find_circuit_from_name(circuit_ensemble,list(freq_dict.keys())[0])

In [37]:
def prep_circuit(input_pauli):
    '''This function prepares the input state for eigenvalue sampling. It takes in a Pauli operator to be input into the circuit and outputs a circuit that prepares the state |psi+> or |psi-> from the input |0>^n, 
    where |psi+> and |psi-> are the eigenstates of the input Pauli operator.
    Inputs: input_pauli - A Qiskit Pauli object or a string representing a Pauli operator.
    Outputs: prep_circ - A Qiskit Circuit object which will prepare the state |psi+> or |psi-> from |0>^n, where |psi+> and |psi-> are the eigenstates of the input Pauli operator.
    '''
    # mapping for preparation circuit. The idea of the prep circuit is that we append it before the circuit to eigenvalue samp
    # i.e. input the state where all qubits are zero 1000...0> into the prep circuit and then input it's ouput into the circu
    # |0> -> ...
    # H -> |+>
    # XH -> |->
    # HS -> |+i>
    # XHS -> |-i>
    # I -> |0> (no change)
    # X -> |1>
    if type(input_pauli) == str:
        input_pauli_string = ''.join(char for char in input_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(input_pauli) == qk.quantum_info.Pauli:
        input_pauli_string = ''.join(char for char in input_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase

    nontriv_indices = [index for index, char in enumerate(input_pauli_string) if char != 'I'] # finds the indices of the nontrivial Pauli gates

    prep_circ = qk.QuantumCircuit(len(input_pauli_string))
    p_eigenstate = [random.choice(range(2)) for i in nontriv_indices] # randomly chooses |psi+> (0) or |psi-> (1)
    
    for index, qubit in enumerate(nontriv_indices):
        pauli_gate =  input_pauli_string[qubit]
        if pauli_gate == "X": # if the input Pauli is X
            if p_eigenstate[index] == 0: # if the randomly chosen eigenvector is |+>
                prep_circ.h(qubit)
            elif p_eigenstate[index] == 1: # if the randomly chosen eigenvector is |->
                prep_circ.x(qubit)
                prep_circ.h(qubit)
        elif pauli_gate == "Y": # if the input Pauli is Y
            if p_eigenstate[index] == 0: # if the randomly chosen eigenvector is l+i>
                prep_circ.h(qubit)
                prep_circ.s(qubit)
            elif p_eigenstate[index] == 1: # if the randomly chosen eigenvector is |-i>
                prep_circ.x(qubit)
                prep_circ.h(qubit)
                prep_circ.s(qubit)
        elif pauli_gate == "Z": # if the input Pauli is Z
            # we don't need to append anything if the chosen eigenstate is 10>, since the input will already be in that state
            if p_eigenstate[index] == 1: # if the randomly chosen eigenvector is
                prep_circ.x(qubit)

    p_in_eigenstate = sum(p_eigenstate) % 2
    return prep_circ, p_in_eigenstate

In [38]:
def measure_circuit(final_pauli):
    if type(final_pauli) == str:
        final_pauli_string = ''.join(char for char in final_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(final_pauli) == qk.quantum_info.Pauli:
        final_pauli_string = ''.join(char for char in final_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase

    nontriv_indices = [index for index, char in enumerate(final_pauli_string) if char != 'I'] # finds the indices of the nontrivial Pauli gates
    nontriv_gates = [final_pauli_string[index] for index in nontriv_indices]

    measurement_circ = qk.QuantumCircuit(len(final_pauli_string),len(nontriv_indices))
    # On the measurement circ, we need to dagger the transforms used to prepare the prep circ to measure in the bases given by P'
    for index, g in enumerate(nontriv_gates):
        # print(index)
        if g == "X":
            measurement_circ.h(nontriv_indices[index]) # rotate into X basis
            measurement_circ.measure(nontriv_indices[index],index) # then measure
        elif g == "Y":
            measurement_circ.sdg(nontriv_indices[index])
            measurement_circ.h(nontriv_indices[index]) # S**-1 H rotates into Y basis
            measurement_circ.measure(nontriv_indices[index],index) # then measure
        elif g == "Z":
            measurement_circ.measure(nontriv_indices[index],index) # measure

    return measurement_circ

In [39]:
st = "XYIZX"
b,c = prep_circuit(st)
print(b)
a = measure_circuit(st)
print(a)
print("Number of Classical Bits: ",a.num_clbits)

     ┌───┐┌───┐     
q_0: ┤ X ├┤ H ├─────
     ├───┤├───┤┌───┐
q_1: ┤ X ├┤ H ├┤ S ├
     └───┘└───┘└───┘
q_2: ───────────────
                    
q_3: ───────────────
     ┌───┐┌───┐     
q_4: ┤ X ├┤ H ├─────
     └───┘└───┘     
      ┌───┐      ┌─┐      
q_0: ─┤ H ├──────┤M├──────
     ┌┴───┴┐┌───┐└╥┘   ┌─┐
q_1: ┤ Sdg ├┤ H ├─╫────┤M├
     └─────┘└───┘ ║    └╥┘
q_2: ─────────────╫─────╫─
       ┌─┐        ║     ║ 
q_3: ──┤M├────────╫─────╫─
       └╥┘  ┌───┐ ║ ┌─┐ ║ 
q_4: ───╫───┤ H ├─╫─┤M├─╫─
        ║   └───┘ ║ └╥┘ ║ 
c: 4/═══╩═════════╩══╩══╩═
        2         0  3  1 
Number of Classical Bits:  4


In [40]:
def est_Lambda(input_pauli,circuit_ensemble,num_experiments):

    if type(input_pauli) == str:
        input_pauli_string = ''.join(char for char in input_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(input_pauli) == qk.quantum_info.Pauli:
        input_pauli_string = ''.join(char for char in input_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase

    freq_dict = random_choices_with_counts([circ.name for circ in circuit_ensemble],num_experiments)

    eigenresults_dict = {("+","+"):0,("+","-"):0,("-","+"):0,("-","-"):0}
    for circ_name in freq_dict:
        circ = find_circuit_from_name(circuit_ensemble,circ_name)
        num_shots = freq_dict[circ_name]
        start_in_plus = random.randint(1, num_shots)
        starter = [start_in_plus, num_shots - start_in_plus]
        
        p_in_eigenstate = 100
        for i in [0,1]:
            while p_in_eigenstate != i:
                prep_circ,p_in_eigenstate = prep_circuit(input_pauli_string)
        
            if p_in_eigenstate == 0:
                p_eigenstate = "+"
            elif p_in_eigenstate == 1:
                p_eigenstate = "-"
            
            prep_circ.barrier()

            measure_circ = measure_circuit(Clifford_Permute(circ,input_pauli_string))

            temp_circuit = prep_circ.compose(circ)
            temp_circuit.barrier()
            full_circuit = temp_circuit.compose(measure_circ)
            # print(full_circuit)
        
            # Simulate the circuit with the stabilizer simulator
            simulator = Aer.get_backend('aer_simulator', method='stabilizer')
            result = qk.execute(full_circuit, backend = simulator, shots = starter[i]).result()
            counts = result.get_counts()

            for r in counts:
                parity = sum(int(x) for x in r if x.isdigit()) % 2
                if parity == 0:
                    pp_eigenstate = "+"
                elif parity == 1:
                    pp_eigenstate= "-"

                if (p_eigenstate,pp_eigenstate) not in eigenresults_dict:
                    eigenresults_dict[(p_eigenstate,pp_eigenstate)] = counts[r]
                else:
                    eigenresults_dict[(p_eigenstate,pp_eigenstate)] += counts[r]

    print(sum(eigenresults_dict.values()))
    # Lambda_est = ((eigenresults_dict[("+","+")] - eigenresults_dict[("+","-")])/(eigenresults_dict[("+","+")] + eigenresults_dict[("+","-")]) - (eigenresults_dict[("-","+")] - eigenresults_dict[("-","-")])/(eigenresults_dict[("-","+")] + eigenresults_dict[("-","-")]))
    Lambda_est = ((eigenresults_dict[("+","+")] - eigenresults_dict[("+","-")]) - (eigenresults_dict[("-","+")] - eigenresults_dict[("-","-")]))/sum(eigenresults_dict.values())

    return Lambda_est

In [41]:
out = est_Lambda(st,[qiskitCircuit],1000)
print(out)

1000
0.012


In [42]:
Clifford_Permute(qiskitCircuit, st)

Pauli('YYXXX')

In [43]:
count_dict = {0:0,1:0}
for i in range(1):
    full_circuit = qk.QuantumCircuit(qiskitCircuit.num_qubits)
    full_circuit.initialize(0)
    
    prep_circ,p_in_eigenstate = prep_circuit(st)
    prep_circ.barrier()
    count_dict[p_in_eigenstate] += 1
    measure_circ = measure_circuit(Clifford_Permute(qiskitCircuit,st))

    temp_circuit = prep_circ.compose(qiskitCircuit)
    temp_circuit.barrier()
    # print(temp_circuit)
    full_circuit = temp_circuit.compose(measure_circ)
    print(full_circuit)
    print("Number of Classical Bits: ", full_circuit.num_clbits)

    simulator = Aer.get_backend('aer_simulator', method='stabilizer')
    result = qk.execute(full_circuit, backend = simulator, shots = 100).result()
    counts = result.get_counts()
    
    print(p_in_eigenstate)
    print(counts)
    print(result)

     ┌───┐┌───┐ ░ ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐ ░ ┌─────┐┌───┐      ┌─┐»
q_0: ┤ X ├┤ H ├─░─┤ X ├─░───────░─┤ S ├─░───────░─┤ S ├─░─┤ Sdg ├┤ H ├──────┤M├»
     ├───┤├───┤ ░ ├───┤ ░ ┌───┐ ░ ├───┤ ░       ░ ├───┤ ░ ├─────┤├───┤      └╥┘»
q_1: ┤ H ├┤ S ├─░─┤ I ├─░─┤ X ├─░─┤ X ├─░───────░─┤ S ├─░─┤ Sdg ├┤ H ├───────╫─»
     └───┘└───┘ ░ ├───┤ ░ └─┬─┘ ░ ├───┤ ░ ┌───┐ ░ ├───┤ ░ └┬───┬┘└┬─┬┘       ║ »
q_2: ───────────░─┤ S ├─░───┼───░─┤ I ├─░─┤ X ├─░─┤ I ├─░──┤ H ├──┤M├────────╫─»
                ░ ├───┤ ░   │   ░ ├───┤ ░ └─┬─┘ ░ ├───┤ ░  ├───┤  └╥┘ ┌─┐    ║ »
q_3: ───────────░─┤ X ├─░───┼───░─┤ S ├─░───■───░─┤ S ├─░──┤ H ├───╫──┤M├────╫─»
     ┌───┐      ░ ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤ ░  ├───┤   ║  └╥┘┌─┐ ║ »
q_4: ┤ H ├──────░─┤ I ├─░───■───░─┤ Z ├─░───────░─┤ Z ├─░──┤ H ├───╫───╫─┤M├─╫─»
     └───┘      ░ └───┘ ░       ░ └───┘ ░       ░ └───┘ ░  └───┘   ║   ║ └╥┘ ║ »
c: 5/══════════════════════════════════════════════════════════════╩═══╩══╩══╩═»
                            

In [44]:
print(full_circuit)
# prep_circ.barrier()
me = prep_circ.compose(qiskitCircuit)
print(me.compose(measure_circ))

     ┌───┐┌───┐ ░ ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐ ░ ┌─────┐┌───┐      ┌─┐»
q_0: ┤ X ├┤ H ├─░─┤ X ├─░───────░─┤ S ├─░───────░─┤ S ├─░─┤ Sdg ├┤ H ├──────┤M├»
     ├───┤├───┤ ░ ├───┤ ░ ┌───┐ ░ ├───┤ ░       ░ ├───┤ ░ ├─────┤├───┤      └╥┘»
q_1: ┤ H ├┤ S ├─░─┤ I ├─░─┤ X ├─░─┤ X ├─░───────░─┤ S ├─░─┤ Sdg ├┤ H ├───────╫─»
     └───┘└───┘ ░ ├───┤ ░ └─┬─┘ ░ ├───┤ ░ ┌───┐ ░ ├───┤ ░ └┬───┬┘└┬─┬┘       ║ »
q_2: ───────────░─┤ S ├─░───┼───░─┤ I ├─░─┤ X ├─░─┤ I ├─░──┤ H ├──┤M├────────╫─»
                ░ ├───┤ ░   │   ░ ├───┤ ░ └─┬─┘ ░ ├───┤ ░  ├───┤  └╥┘ ┌─┐    ║ »
q_3: ───────────░─┤ X ├─░───┼───░─┤ S ├─░───■───░─┤ S ├─░──┤ H ├───╫──┤M├────╫─»
     ┌───┐      ░ ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤ ░  ├───┤   ║  └╥┘┌─┐ ║ »
q_4: ┤ H ├──────░─┤ I ├─░───■───░─┤ Z ├─░───────░─┤ Z ├─░──┤ H ├───╫───╫─┤M├─╫─»
     └───┘      ░ └───┘ ░       ░ └───┘ ░       ░ └───┘ ░  └───┘   ║   ║ └╥┘ ║ »
c: 5/══════════════════════════════════════════════════════════════╩═══╩══╩══╩═»
                            

In [45]:
def get_unitary(circuit):
    simulator = Aer.get_backend('unitary_simulator')
    t_circuit = transpile(circuit, simulator)
    qobj = assemble(t_circuit)
    result = simulator.run(qobj)
    return result.get_unitary(circuit, decimals=3)

# Get unitary matrices for each circuit
unitaries = [Clifford_Permute(circuit,qk.quantum_info.Pauli("XYIZX")) for circuit in circuit_ensemble]

# Check if all unitaries are equal
are_equal = all(np.allclose(unitaries[0], unitary) for unitary in unitaries[1:])

if are_equal:
    print("The unitaries of all circuits are equal.")
else:
    print("The unitaries of the circuits are not equal.")

The unitaries of all circuits are equal.


In [64]:
circuit = generateCliffordCircuit(2, 2, singleGateSet, doubleGateSet)
meme = transpileListToQiskitCircuit(circuit)
print(meme)
from qiskit.transpiler.passes import RemoveBarriers
rb = RemoveBarriers()
meme2 = rb(meme)
print(meme2)

     ┌───┐ ░      
q_0: ┤ I ├─░───■──
     ├───┤ ░ ┌─┴─┐
q_1: ┤ I ├─░─┤ X ├
     └───┘ ░ └───┘
     ┌───┐     
q_0: ┤ I ├──■──
     ├───┤┌─┴─┐
q_1: ┤ I ├┤ X ├
     └───┘└───┘


In [77]:
circuit = generateCliffordCircuit(2, 2, singleGateSet, doubleGateSet)
meme = transpileListToQiskitCircuit(circuit)
print(meme)
p_str = "YX"
print(Clifford_Permute(meme, p_str).to_label())
v = est_Lambda(p_str,[meme],1000)
print(v)

     ┌───┐ ░ ┌───┐
q_0: ┤ X ├─░─┤ X ├
     ├───┤ ░ └─┬─┘
q_1: ┤ Z ├─░───■──
     └───┘ ░      
-YI
1000
-0.022


In [119]:
circuit = generateCliffordCircuit(2, 2, singleGateSet, doubleGateSet)
meme = transpileListToQiskitCircuit(circuit)
layers = [x for x in split_circuit_by_barrier(meme)]
print(layers[0])
print(Clifford_Permute(layers[0], p_str))
v = est_Lambda(p_str,[layers[0]],1000)
print(v)

     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ H ├
     └───┘
-YX
1000
0.002


In [133]:
a = qk.QuantumCircuit(1)
a.x(0)
print(Clifford_Permute(a,"Y"))

a = qk.QuantumCircuit(1)
a.h(0)
print(Clifford_Permute(a,"X"))

-Y
Z


In [131]:
a = qk.QuantumCircuit(2)
a.x(0)
a.h(1)
a.draw()
p = qk.quantum_info.Pauli("YX")
p.evolve(a, qargs =[0,1], frame="s")

Pauli('-YX')

In [134]:
singleGateSet

['X', 'H', 'Z', 'I', 'S']

In [141]:
for g in singleGateSet:
    a = qk.QuantumCircuit(1)
    if g == "X":
        a.x(0)
    elif g == "Y":
        a.y(0)
    elif g == "Z":
        a.z(0)
    elif g == "H":
        a.h(0)
    elif g == "S":
        a.s(0)
    elif g == "I":
        a.id(0)
    print(Clifford_Permute(a,"Z"))
    print(est_Lambda("Y",[a],1000))
    print()

-Z
1000
-1.0

X
1000
-1.0

Z
1000
-1.0

Z
1000
1.0

Z
1000
-1.0

